# Objective: Scrape everything on the page
 - creating breif of each legislator that contains sentator and rep info for each distict (for a 3rd party)
     - legislative name/contact info
     - picture
     - capital office location
     - district office location 
     - JJ usually prints a page ith senator info on front, and rep on back
        

## First: Obtain the list of links to each Senator's webpage

In [1]:
# import dependacies 

from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# set up the chrome driever
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# IL Genearl Assembly homepage
url = 'http://www.ilga.gov'

In [4]:
# retrieve the webpage, create BeautifulSoup object to parse the data
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

the tabel is in the body of the page
    this tabel is in the td of another table
  
    this div is the td of a table
    <div>
    <table>
    <tbody>
    <tr>
    <td>
    <a class="homemenu" href="/senate/" title="View the biography, sponsored bills and committees for each Senate member">
                Members

In [5]:
# Locate link to all Senators and go to the page 
links = soup.find_all(href=re.compile("/senate/"), text='Members')

for link in links:
    reference = link.get('href')
#     browser.click_link_by_partial_text('Members')
    senate_url = url+reference
print(senate_url)
browser.visit(senate_url)
html = browser.html
senate_soup = BeautifulSoup(html, 'html.parser')
# print(senate_soup.prettify())

http://www.ilga.gov/senate/


In [6]:
# for link in senate_soup.find_all('a'):
#     print(link.get('href'))

senator_links = senate_soup.find_all('a', {'class':'notranslate'})
# senate_soup.find_all('a')

# create empty list to hold all member urls
senator_link_list = []

# loop though senate page and add each member link to the senator_link_list
for link in senator_links:
    reference = link.get('href')
#     browser.click_link_by_partial_text('Members')
    senator_url = url+reference
    senator_link_list.append(senator_url)
    
# print(senator_link_list[0])

In [7]:
# close the window
browser.quit()

#  save the list 



### Work with the link to the 1st senator's page, create series of functions to obtain the information on thier page. 

In [ ]:
# visit the first senator page

browser.visit(senator_link_list[0])
html = browser.html
senator_soup = BeautifulSoup(html, 'html.parser')
# print(senator_soup.prettify)

In [ ]:
# Obtain the senator's and district number
senator_name = senator_soup.find('span', {'class':'notranslate'}).text
senator_district = senator_soup.find('span', {'class':'heading2 notranslate'}).text

print(senator_name)
print(senator_district)

In [ ]:
# Obtain the link to the image of the senator 

# Obtain the parital image link from the senator's webpage
senator_img_partial = senator_soup.find('table', {'class' : 'notranslate'}).img['src']
# senator_img_partial


# obtain the full image link by combining the orignal il.gov url with the partial senator image link
senator_img = url + senator_img_partial
senator_img

In [ ]:
# Get the leadership name
senator_position_all = senator_soup.find_all('span', {'class':'heading2'})
senator_position = senator_position_all[3].text
senator_position

In [ ]:
# Most of the sentor info are in tables, so can gather the address and committee assignments from td in the "member" class
senator_info_list = senator_soup.find_all('td', {'class': 'member'})
senator_info_list

In [ ]:
# Springfield office information

senator_springfield_address_line1 = senator_info_list[1].text
# senator_springfield_address_line1
senator_springfield_address_line2 = senator_info_list[2].text
senator_springfield_address_line3 = senator_info_list[3].text

senator_springfield_phone_number = senator_info_list[4].text


In [ ]:
#  District office information

senator_district_address_line1 = senator_info_list[8].text

senator_district_address_line2 = senator_info_list[9].text
senator_district_address_line3 = senator_info_list[10].text

senator_district_phone_number = senator_info_list[11].text

# senator_district_address_line1

In [ ]:
# Senator info: The # of years served, committee assignments, and biographhy are all under the same child tag

senator_bio_info  = senator_info_list[12].text
senator_bio_info

In [ ]:
ass_reps_tag = senator_soup.find_all('a', {'class': 'notranslate'})
ass_reps_tag

In [ ]:
current_senator_assoc_reps = []
for tag in ass_reps_tag:
    rep = tag.text
    current_senator_assoc_reps.append(rep)
#     print(rep)
# current_senator_assoc_reps

In [ ]:
# close the window
browser.quit()

## Next: Create a function that will go to each senator's page and scrape thier information.

In [8]:
# Create a function that will go to each senator's page and scrape thier information


def senator_scrape(link):

    # visit the first senator page
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(link)
    html = browser.html
    senator_soup = BeautifulSoup(html, 'html.parser')


    
    # Obtain the senator's and district number
    senator_name = senator_soup.find('span', {'class':'notranslate'}).text
    senator_district = senator_soup.find('span', {'class':'heading2 notranslate'}).text


    # Obtain the parital image link from the senator's webpage
    senator_img_partial = senator_soup.find('table', {'class' : 'notranslate'}).img['src']


    # obtain the full image link by combining the orignal il.gov url with the partial senator image link
    senator_img = url + senator_img_partial


    # Get the leadership position
    senator_position_all = senator_soup.find_all('span', {'class':'heading2'})
    try:
        senator_position = senator_position_all[3].text
        
    except: 
        senator_position = 'none'
        print('No position exists')


    # Most of the sentor info are in tables, so can gather the address and committee assignments from td in the "member" class
    senator_info_list = senator_soup.find_all('td', {'class': 'member'})


    # Springfield office information
    senator_springfield_address_line1 = senator_info_list[1].text
    senator_springfield_address_line2 = senator_info_list[2].text
    senator_springfield_address_line3 = senator_info_list[3].text

    senator_springfield_phone_number = senator_info_list[4].text


    #  District office information
    senator_district_address_line1 = senator_info_list[8].text
    senator_district_address_line2 = senator_info_list[9].text
    senator_district_address_line3 = senator_info_list[10].text

    senator_district_phone_number = senator_info_list[11].text

  
    # Senator info: The # of years served, committee assignments, and biographhy are all under the same child tag
    senator_bio_info  = senator_info_list[12].text

    assoc_reps_tag = senator_soup.find_all('a', {'class': 'notranslate'})



    current_senator_assoc_reps = []
    for tag in assoc_reps_tag:
        rep = tag.text
        current_senator_assoc_reps.append(rep)


    all_senator_details = {
        'name': senator_name,
        'district': senator_district,
        'picture': senator_img,
        'role': senator_position,

        'spg_address_line1': senator_springfield_address_line1,
        'spg_address_line2': senator_springfield_address_line2,
        'spg_address_line3': senator_springfield_address_line3,
        'spg_phone_number': senator_springfield_phone_number,

        'dst_address_line1': senator_district_address_line1,
        'dst_address_line2': senator_district_address_line2,
        'dst_address_line3': senator_district_address_line3,
        'dst_phne_number' : senator_district_phone_number,
        
        'bio' : senator_bio_info,
        
        'associate_reps' : current_senator_assoc_reps

    }

    # close the window
    browser.quit()

    return all_senator_details

In [ ]:
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)
# link = senator_link_list[1]
# senator_scrape(link)

### Test the scrape function

In [9]:
scraped_senator_details = senator_scrape(senator_link_list[0])

In [10]:
scraped_senator_details

{'name': 'Neil Anderson (R)',
 'district': '36th',
 'picture': 'http://www.ilga.gov/images/members/{90CDA259-1DEA-4D18-AE97-30051E03D154}.jpg',
 'role': 'Assistant Republican Leader',
 'spg_address_line1': 'Senator 36th District',
 'spg_address_line2': 'M103D  Capitol Building',
 'spg_address_line3': 'Springfield, IL \xa0\xa062706    ',
 'spg_phone_number': '(217) 782-5957',
 'dst_address_line1': '1523 47th Ave.',
 'dst_address_line2': 'Suite 2',
 'dst_address_line3': 'Moline, IL\xa0\xa061265    ',
 'dst_phne_number': '(309) 736-7084',
 'bio': 'Years served:  Committee assignments: Committee of the Whole; Commerce and Economic Development; Energy and Public Utilities; Licensed Activities (Minority Spokesperson); Transportation; Veterans Affairs; Subcommittee on Capital (TR); Oversight Medicaid Mang. Care, Spec; Opioid Crisis Abatement Spec. Com..Biography: ',
 'associate_reps': ['Tony McCombie', 'Michael Halpin']}

## Use the function to grab information about each senator, from thier website

In [12]:
senator_information = {}
senator_number = 0

for link in senator_link_list:
    try: 
        print('---------------------------')
        print(f'Trying senator {senator_number}')
        
        senator_scrape(link)
        
        senator_information.update(scraped_senator_details)
        
    except: 
        print(f'There are errors with {senator_number}')
        
    senator_number += 1
    
    if senator_number == 3 :
        break
        
#     senator_information[all_senator_details['name']] = all_senator_details
    # senator_number = senator_number + 1

---------------------------
Trying senator 0
---------------------------
Trying senator 1
No position exists
---------------------------
Trying senator 2


### There are errors with senators 11, 15, 41, 43, 48, 51

In [13]:
senator_information

{'name': 'Neil Anderson (R)',
 'district': '36th',
 'picture': 'http://www.ilga.gov/images/members/{90CDA259-1DEA-4D18-AE97-30051E03D154}.jpg',
 'role': 'Assistant Republican Leader',
 'spg_address_line1': 'Senator 36th District',
 'spg_address_line2': 'M103D  Capitol Building',
 'spg_address_line3': 'Springfield, IL \xa0\xa062706    ',
 'spg_phone_number': '(217) 782-5957',
 'dst_address_line1': '1523 47th Ave.',
 'dst_address_line2': 'Suite 2',
 'dst_address_line3': 'Moline, IL\xa0\xa061265    ',
 'dst_phne_number': '(309) 736-7084',
 'bio': 'Years served:  Committee assignments: Committee of the Whole; Commerce and Economic Development; Energy and Public Utilities; Licensed Activities (Minority Spokesperson); Transportation; Veterans Affairs; Subcommittee on Capital (TR); Oversight Medicaid Mang. Care, Spec; Opioid Crisis Abatement Spec. Com..Biography: ',
 'associate_reps': ['Tony McCombie', 'Michael Halpin']}

In [14]:
senator_df = pd.DataFrame(senator_information)

In [15]:
senator_df

,name,district,picture,role,spg_address_line1,spg_address_line2,spg_address_line3,spg_phone_number,dst_address_line1,dst_address_line2,dst_address_line3,dst_phne_number,bio,associate_reps
0,Neil Anderson (R),36th,http://www.ilga.gov/images/members/{90CDA259-1...,Assistant Republican Leader,Senator 36th District,M103D Capitol Building,"Springfield, IL 62706",(217) 782-5957,1523 47th Ave.,Suite 2,"Moline, IL 61265",(309) 736-7084,Years served: Committee assignments: Committe...,Tony McCombie
1,Neil Anderson (R),36th,http://www.ilga.gov/images/members/{90CDA259-1...,Assistant Republican Leader,Senator 36th District,M103D Capitol Building,"Springfield, IL 62706",(217) 782-5957,1523 47th Ave.,Suite 2,"Moline, IL 61265",(309) 736-7084,Years served: Committee assignments: Committe...,Michael Halpin
